In [20]:
import pandas as pd

# Chemin du fichier Excel
file_path = "../sources/rapport_banque_fictif_casse_gueule.xlsx"

# Lire le fichier Excel
df = pd.read_excel(file_path)

# Afficher les premières lignes du tableau
print(df)

# Exploiter les données (exemple : calculer la moyenne du chiffre d'affaires)
moyenne_chiffre_affaires = df["Chiffre d Affaires (k€)"].mean()
print(f"Moyenne du chiffre d'affaires (en k€) : {moyenne_chiffre_affaires:.2f}")

# Transformer le tableau en dictionnaire si besoin
data_dict = df.to_dict(orient="records")
print(data_dict)


     Banque  Nouveaux Clients  Chiffre d Affaires (k€)  \
0  Banque A                45                    123.4   
1  Banque B                78                    156.7   
2  Banque C                 2                      0.2   
3  Banque D                56                    134.9   
4  Banque E                89                    176.3   

   Taux d Apport en Assurance Vie (%)  Nombre de Contrats Signés  \
0                                12.4                         25   
1                                15.3                         40   
2                                 0.4                          1   
3                                14.1                         35   
4                                16.7                         48   

   Nombre de Dossiers de Crédit Approuvés  Réclamations Clients  
0                                      15                     2  
1                                      28                     3  
2                                       0   

In [19]:
import openai
import os
from dotenv import load_dotenv
import pandas as pd

# Charger la clé API depuis le fichier .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Fonction pour envoyer une requête avec un pre-prompt
def ask_gpt_with_preprompt(pre_prompt, user_query, model="gpt-4-0613", max_tokens=300):
    try:
        # Messages comprenant le pre-prompt et la question utilisateur
        messages = [
            {"role": "system", "content": pre_prompt},  # Pre-prompt
            {"role": "user", "content": user_query}    # Question utilisateur
        ]
        
        # Requête à l'API
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7  # Ajuste la créativité du modèle
        )
        
        # Retourne la réponse du modèle
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Erreur lors de l'appel à l'API : {e}"

# Pre-prompt pour guider GPT-4
pre_prompt = (
    "Tu es un analyste financier spécialisé dans le domaine bancaire. "
    "Tu vas analyser les données hebdomadaires pour une banque spécifique et rédiger un compte rendu clair et synthétique. "
    "Inclue les performances en termes de nouveaux clients, chiffre d'affaires, assurance vie, contrats signés et réclamations clients."
)

# Générer un compte rendu pour chaque banque
def generate_reports(df):
    reports = {}
    for index, row in df.iterrows():
        # Assignation temporaire des valeurs pour éviter les conflits avec les f-strings
        bank_name = row["Banque"]
        nouveaux_clients = row["Nouveaux Clients"]
        chiffre_affaires = row["Chiffre d Affaires (k€)"]
        taux_assurance_vie = row["Taux d Apport en Assurance Vie (%)"]
        contrats_signes = row["Nombre de Contrats Signés"]
        credits_approuves = row["Nombre de Dossiers de Crédit Approuvés"]
        reclamations_clients = row["Réclamations Clients"]

        # Construire la question utilisateur à partir des données de la banque
        user_query = (
            f"Voici les données de la banque {bank_name} cette semaine :\n"
            f"- Nouveaux clients : {nouveaux_clients}\n"
            f"- Chiffre d affaires : {chiffre_affaires} k€\n"
            f"- Taux d apport en assurance vie : {taux_assurance_vie}%\n"
            f"- Nombre de contrats signés : {contrats_signes}\n"
            f"- Nombre de dossiers de crédit approuvés : {credits_approuves}\n"
            f"- Réclamations clients : {reclamations_clients}\n"
            "Rédige un compte rendu détaillé pour cette banque."
        )
        
        # Appeler GPT pour générer le compte rendu
        report = ask_gpt_with_preprompt(pre_prompt, user_query)
        reports[bank_name] = report
    
    return reports

# Générer les comptes rendus
reports = generate_reports(df)

# Afficher les comptes rendus
for bank, report in reports.items():
    print(f"\nCompte rendu pour {bank} :\n{report}\n")



Compte rendu pour Banque A :
Compte rendu hebdomadaire pour la Banque A

Au cours de la semaine dernière, la Banque A a connu une croissance modérée en termes de nouveaux clients. Un total de 45 nouveaux clients ont été enregistrés, ce qui est une indication positive de la performance de la banque en termes d'acquisition de clients. 

Le chiffre d'affaires généré pour la semaine s'élève à 123,4 k€. C'est un indicateur important qui mesure la performance monétaire globale de la banque. Il est essentiel de continuer à surveiller ce chiffre pour évaluer la santé financière de l'entreprise.

En ce qui concerne l'assurance vie, le taux d'apport a été de 12,4%. Cela donne une indication de la proportion de clients qui ont souscrit à l'assurance vie par rapport au total des clients de la banque. Il serait bénéfique de continuer à promouvoir ce produit afin d'augmenter ce pourcentage.

Sur le plan des contrats, la banque a réussi à en signer 25 durant la semaine. Cela inclut probablement une 